In [1]:
import re, nltk, string, warnings 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt  

pd.set_option("display.max_colwidth", 200) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline

In [2]:
train  = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv')

In [3]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5279 entries, 0 to 5278
Data columns (total 4 columns):
unique_hash    5279 non-null object
text           5279 non-null object
drug           5279 non-null object
sentiment      5279 non-null int64
dtypes: int64(1), object(3)
memory usage: 165.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924 entries, 0 to 2923
Data columns (total 3 columns):
unique_hash    2924 non-null object
text           2924 non-null object
drug           2924 non-null object
dtypes: object(3)
memory usage: 68.6+ KB


In [4]:
drugs=list(set(list(train['drug'].unique())+list(test['drug'].unique())))

In [5]:
train['tidy_text'] = train['text'].str.replace("[^a-zA-Z#]", " ")
train['tidy_text'] = train['tidy_text'].str.lower()
train['tidy_text'] = train['tidy_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [6]:
from nltk.corpus import stopwords
stop=stopwords.words('english')

In [7]:
stop=stop+drugs
stopword = r'\b(?:{})\b'.format('|'.join(stop))
train['tidy_text'] = train['tidy_text'].str.replace(stopword, '')

In [8]:
train.head(10)

,unique_hash,text,drug,sentiment,tidy_text
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,"Autoimmune diseases tend to come in clusters. As for Gilenya – if you feel good, don’t think about it, it won’t change anything but waste your time and energy. I’m taking Tysabri and feel amazing,...",gilenya,2,autoimmune diseases tend come clusters feel good think change anything waste time energy taking feel amazing symptoms dodgy color vision since always know know last month year decade decid...
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,"I can completely understand why you’d want to try it. But, results reported in lectures don’t always stand up to the scrutiny of peer-review during publication. There so much still to do before th...",gilenya,2,completely understand want results reported lectures always stand scrutiny peer review publication much still convincing hope work really aware happy risks great think important present ...
2,fe809672251f6bd0d986e00380f48d047c7e7b76,"Interesting that it only targets S1P-1/5 receptors rather than 1-5 like Fingolimod. Hoping to soon see what the AEs and SAEs were Yes. I'm not sure what this means, exactly: Quote Nine patients r...",fingolimod,2,interesting targets receptors rather like hoping soon saes sure means exactly quote nine patients reported serious adverse events serious adverse event reported patient safety signals oc...
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder where lemtrada and ocrevus sales would go, if they prove anti-cd20 are induction",ocrevus,2,interesting grand merci wonder sales would prove anti induction
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain and Cervical Cord are in and my next Neurologist appointment is in the next couple of weeks. There’re no new lesions in Brain/Cord and I’ve had no rel...",gilenya,1,everybody latest results brain cervical cord next neurologist appointment next couple weeks lesions brain cord relapses good sign line cervical cord review concerned goes lesions show hypoin...
5,a043780c757966243779bf3c0d11bf6eef721971,I can’t give you advice about Lemtrada because I chose Cladribine. Have you thought about this drug? The doctors at Barts are keen to give it to people with SPMS. You can read about it here: http:...,cladribine,2,give advice chose thought drug doctors barts keen give people spms read http multiple sclerosis research blogspot suppose therapy html
6,be5a13376933a7f9bbf8e801c31691092f63260a,Reply posted for JessZidek. Hi Jess Sorry to read about the challenges you are having with your health. You mentioned a lot in your post. I just want to share some info on a few of the points. Fir...,humira,0,reply posted jesszidek jess sorry read challenges health mentioned post want share info points first know said scared biologics successful reducing symptoms inducing maintain disease r...
7,08c3c0c702fc97d290204b37798ac62005da5626,"Well as expected my Neurologist wants me to start Tysabri, I kept saying that I wasn’t happy and he kept saying Yes you are! But I am still NOT, If Lemtrada was available here I think I would defi...",gilenya,2,well expected neurologist wants start kept saying happy kept saying still available think would definitely like every thing behind took years longer australia taking every second vitals...
8,8fd3d7ad80791c9343e5cf8a83bd1adf6577d516,"Why do you think that FIngolimod was such a miserable failure in progressive MS trial in humans (not animals) that was aborted by Biogen? If it is in fact stimulating neuronal gene expression, axo...",fingolimod,1,think miserable failure progressive trial humans animals aborted biogen fact stimulating neuronal gene expression axon growth regeneration want progressive patients human trials fail
9,793c5af7cc8332df17eb602247d886fbd1c80f89,"Thank you so much…I’m learning a lot here at GRACE. I should have mentioned my husband’s cancer is in his bones, liver, adrenal, in addition to lung and brain mets a

In [9]:
train_token = train['tidy_text'].apply(lambda x: x.split()) # tokenizing 
train_token.head()

0    [autoimmune, diseases, tend, come, clusters, feel, good, think, change, anything, waste, time, energy, taking, feel, amazing, symptoms, dodgy, color, vision, since, always, know, know, last, month...
1    [completely, understand, want, results, reported, lectures, always, stand, scrutiny, peer, review, publication, much, still, convincing, hope, work, really, aware, happy, risks, great, think, impo...
2    [interesting, targets, receptors, rather, like, hoping, soon, saes, sure, means, exactly, quote, nine, patients, reported, serious, adverse, events, serious, adverse, event, reported, patient, saf...
3                                                                                                                                  [interesting, grand, merci, wonder, sales, would, prove, anti, induction]
4    [everybody, latest, results, brain, cervical, cord, next, neurologist, appointment, next, couple, weeks, lesions, brain, cord, relapses, good, sign, line, cervical, cord, revi

In [10]:
train_tok=train_token.tolist()

In [11]:
all_words=[]
def nest_rem(l): 
    for i in l: 
        if type(i) == list: 
            nest_rem(i) 
        else: 
            all_words.append(i) 
nest_rem(train_tok)

In [12]:
len(all_words)

883708

In [13]:
len(list(set(all_words)))

34161

In [17]:
test['tidy_text'] = test['text'].str.replace("[^a-zA-Z#]", " ")
test['tidy_text'] = test['tidy_text'].str.lower()
test['tidy_text'] = test['tidy_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
test['tidy_text'] = test['tidy_text'].str.replace(stopword, '')